In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import seaborn as sns
pd.set_option('display.max_columns', None)

# Load Data
df = pd.read_excel("stockist_data_with_date3.xlsx")

In [2]:
df.shape

(30000, 31)

In [3]:
df.head()

,Partner_id,AIS(Air Insulated Switchgear),RMU(Ring Main Unit),PSS(Compact Sub-Stations),VCU(Vacuum Contactor Units),E-House,VCB(Vacuum Circuit Breaker),ACB(Air Circuit Breaker),MCCB(Moduled Case Circuit Breaker),SDF(Switch Disconnectors),BBT(Busbar Trunking),Modular Switches,Starter,Controller,Solar Solutions,Pump Starter and Controller,Geography,Stockist_Type,Scheme_Type,Sales_Value_Last_Period,Sales_Quantity_Last_Period,MRP,Discount_Applied,Bulk_Purchase_Tendency,Competitor_Price,Competitor_Discount,Date,Month,Year,Sales_Value_Last_Period_3M_Ago,Discount_Applied_3M_Ago
0,281_74,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,West,Retailer,Bulk Purchase,22014,2,4679,5,0,914.92,7,2024-12-10,12,2024,18426.25864,0
1,549_85,0,1,0,1,1,1,0,1,0,1,1,1,0,1,1,North,Wholesaler,Bulk Purchase,28863,2,3335,7,0,915.17,9,2025-01-04,1,2025,25648.28893,6
2,742_66,1,0,0,0,0,1,0,1,1,1,1,1,1,0,1,Central,Wholesaler,Discount,13729,2,1065,2,1,917.96,0,2022-10-23,10,2022,16393.81271,0
3,199_92,1,0,0,0,1,0,1,1,0,0,1,1,1,1,1,Central,Retailer,Bulk Purchase,22038,2,1244,5,1,918.68,1,2022-09-17,9,2022,25221.93771,0
4,412_67,1,1,0,1,0,1,1,1,0,0,1,0,0,1,0,Central,Wholesaler,Loyalty,26075,2,4604,2,0,919.33,3,2023-03-23,3,2023,27523.66971,0


In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split

# Ensure Date column is properly converted to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # 'coerce' will convert invalid dates to NaT

# Drop rows with invalid dates if any
df = df.dropna(subset=['Date'])

# Sort by date
df = df.sort_values(by='Date')

# Extract relevant product categories (using your original column names)
expected_product_columns = [
    'AIS(Air Insulated Switchgear)', 'RMU(Ring Main Unit)', 
    'PSS(Compact Sub-Stations)', 'VCU(Vacuum Contactor Units)', 
    'E-House', 'VCB(Vacuum Circuit Breaker)', 
    'ACB(Air Circuit Breaker)', 'MCCB(Moduled Case Circuit Breaker)', 
    'SDF(Switch Disconnectors)', 'BBT(Busbar Trunking)', 
    'Modular Switches', 'Starter', 'Controller', 
    'Solar Solutions', 'Pump Starter and Controller'
]

# Find which of these columns actually exist in the dataframe
product_columns = [col for col in expected_product_columns if col in df.columns]

# Create Product_id based on which product has value 1
df['Product_id'] = df[product_columns].idxmax(axis=1)

# Selecting necessary columns
df = df[['Partner_id', 'Product_id', 'Date', 'MRP', 'Sales_Quantity_Last_Period', 
         'Discount_Applied', 'Geography', 'Competitor_Price']].dropna()

# Rename columns
df.rename(columns={
    'MRP': 'Price', 
    'Sales_Quantity_Last_Period': 'Demand', 
    'Discount_Applied': 'Discount'
}, inplace=True)

# Filter valid data
df = df[(df['Price'] > 0) & (df['Demand'] > 0)]

In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split


def prepare_data(df):
    """Prepare and clean the input dataframe with proper date handling"""
    # Convert and validate dates
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date']).sort_values('Date')
    
    # Create proper monthly index (ensuring correct period conversion)
    df['YearMonth'] = pd.to_datetime(df['Date'].dt.to_period('M').astype(str))

    # Set as datetime index
    df.set_index('YearMonth', inplace=True)


    
    # Aggregate to monthly level
    monthly_df = df.groupby(['Product_id', 'Geography', 'YearMonth']).agg({
        'Demand': 'sum',
        'Price': 'mean',
        'Competitor_Price': 'mean',
        'Discount': 'mean'
    }).reset_index()
    
    # Log transformations with proper handling
    for col in ['Demand', 'Price', 'Competitor_Price']:
        monthly_df[f'Log_{col}'] = np.log1p(monthly_df[col])  # Using log1p to handle zeros
    
    return monthly_df.dropna()

In [6]:
df

,Partner_id,Product_id,Date,Price,Demand,Discount,Geography,Competitor_Price
22601,22602_67,RMU(Ring Main Unit),2020-03-05,4227,76,3,East,4195.81
2227,02228_86,VCU(Vacuum Contactor Units),2020-03-05,3432,8,4,North,1360.15
29628,29629_69,AIS(Air Insulated Switchgear),2020-03-05,4793,99,5,South,5597.62
5021,05022_87,AIS(Air Insulated Switchgear),2020-03-05,2588,23,25,South,1754.46
23931,23932_27,RMU(Ring Main Unit),2020-03-05,1937,80,9,South,4390.01
...,...,...,...,...,...,...,...,...
11350,11351_89,PSS(Compact Sub-Stations),2025-03-04,4773,39,5,South,2624.43
4532,04533_23,AIS(Air Insulated Switchgear),2025-03-04,4997,23,7,South,1686.44
3496,03497_50,VCU(Vacuum Contactor Units),2025-03-04,3023,22,5,East,1538.38
8523,08524_58,AIS(Air Insulated Switchgear),2025-03-04,3934,29,6,East,2232.02


In [7]:
# Prepare the data
monthly_df = prepare_data(df)

In [8]:
monthly_df

,Product_id,Geography,YearMonth,Demand,Price,Competitor_Price,Discount,Log_Demand,Log_Price,Log_Competitor_Price
0,ACB(Air Circuit Breaker),Central,2020-03-01,35,2394.000000,2470.590000,9.000000,3.583519,7.781139,7.812617
1,ACB(Air Circuit Breaker),Central,2020-04-01,53,4197.000000,3252.380000,25.000000,3.988984,8.342364,8.087450
2,ACB(Air Circuit Breaker),Central,2020-05-01,67,1836.000000,3824.290000,8.000000,4.219508,7.515889,8.249390
3,ACB(Air Circuit Breaker),Central,2020-06-01,76,1125.000000,4190.790000,7.000000,4.343805,7.026427,8.340883
4,ACB(Air Circuit Breaker),Central,2020-07-01,88,2453.500000,2858.535000,6.000000,4.488636,7.805678,7.958414
...,...,...,...,...,...,...,...,...,...,...
2099,VCU(Vacuum Contactor Units),West,2024-10-01,429,3509.500000,3257.976250,12.625000,6.063785,8.163514,8.089168
2100,VCU(Vacuum Contactor Units),West,2024-11-01,365,3901.428571,3187.494286,11.428571,5.902633,8.269354,8.067304
2101,VCU(Vacuum Contactor Units),West,2024-12-01,276,2903.500000,2419.586250,4.250000,5.624018,7.974017,7.791765
2102,VCU(Vacuum Contactor Units),West,2025-01-01,310,2945.800000,3605.706000,4.400000,5.739793,7.988475,8.190550


In [9]:
monthly_df

,Product_id,Geography,YearMonth,Demand,Price,Competitor_Price,Discount,Log_Demand,Log_Price,Log_Competitor_Price
0,ACB(Air Circuit Breaker),Central,2020-03-01,35,2394.000000,2470.590000,9.000000,3.583519,7.781139,7.812617
1,ACB(Air Circuit Breaker),Central,2020-04-01,53,4197.000000,3252.380000,25.000000,3.988984,8.342364,8.087450
2,ACB(Air Circuit Breaker),Central,2020-05-01,67,1836.000000,3824.290000,8.000000,4.219508,7.515889,8.249390
3,ACB(Air Circuit Breaker),Central,2020-06-01,76,1125.000000,4190.790000,7.000000,4.343805,7.026427,8.340883
4,ACB(Air Circuit Breaker),Central,2020-07-01,88,2453.500000,2858.535000,6.000000,4.488636,7.805678,7.958414
...,...,...,...,...,...,...,...,...,...,...
2099,VCU(Vacuum Contactor Units),West,2024-10-01,429,3509.500000,3257.976250,12.625000,6.063785,8.163514,8.089168
2100,VCU(Vacuum Contactor Units),West,2024-11-01,365,3901.428571,3187.494286,11.428571,5.902633,8.269354,8.067304
2101,VCU(Vacuum Contactor Units),West,2024-12-01,276,2903.500000,2419.586250,4.250000,5.624018,7.974017,7.791765
2102,VCU(Vacuum Contactor Units),West,2025-01-01,310,2945.800000,3605.706000,4.400000,5.739793,7.988475,8.190550


In [14]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_test_split_monthly(df, test_size=0.2):
    """
    Time-based train-test split that maintains temporal order
    """
    # Convert to datetime if not already
    df['YearMonth'] = pd.to_datetime(df['YearMonth'])
    
    # Get all unique months and sort them
    unique_months = pd.Series(df['YearMonth'].unique()).sort_values().values
    
    # Calculate split point
    split_idx = int(len(unique_months) * (1 - test_size))
    train_months = unique_months[:split_idx]
    test_months = unique_months[split_idx:]
    
    # Split the data
    train_df = df[df['YearMonth'].isin(train_months)]
    test_df = df[df['YearMonth'].isin(test_months)]
    
    return train_df, test_df



# from statsmodels.tsa.statespace.sarimax import SARIMAX

# def train_sarimax_model(train_group):
#     """Train SARIMAX model on training data only"""
#     try:
#         # Prevent modifying original data
#         train_group = train_group.copy().sort_values('YearMonth')

#         print("Min values:\n", train_group[['Log_Demand', 'Log_Price', 'Log_Competitor_Price']].min())
#         print("Max values:\n", train_group[['Log_Demand', 'Log_Price', 'Log_Competitor_Price']].max())
#         print("Check for NaN:\n", train_group.isna().sum())
        

#         # Ensure proper index format
#         train_group['YearMonth'] = pd.to_datetime(train_group['YearMonth'])
#         train_group.set_index('YearMonth', inplace=True)
#         # train_group = train_group.resample('MS').asfreq()
        
#         train_group.index = pd.date_range(start=train_group.index.min(), 
#                                   periods=len(train_group), 
#                                   freq='MS')

#         # Extract target variable
#         y_train = train_group['Log_Demand'].dropna()
#         print("Unique time periods:", len(y_train))

#         # Extract exogenous variables (ensure no missing values)
#         exog_cols = ['Log_Price', 'Log_Competitor_Price']
#         exog_train = train_group[exog_cols].dropna()

#         # Skip if insufficient training data
#         if len(y_train) < 24:
#             print("Skipping training: Insufficient data.")
#             return None

#         # Train SARIMAX model
#         model = SARIMAX(
#             y_train,
#             exog=exog_train if not exog_train.empty else None,  # Handle case where exog might be empty
#             order=(1, 1, 1),
#             seasonal_order=(0, 1, 1, 6),
#             enforce_stationarity=False,
#             enforce_invertibility=False 
#         )
#         # fitted_model = model.fit(disp=False)
#         fitted_model = model.fit(method='powell', disp=False)


#         return fitted_model

#     except Exception as e:
#         print(f"Training failed: {str(e)}")
#         return None


def train_sarimax_model(train_group):
    """Train SARIMAX model on training data only"""
    try:
        # Prevent modifying original data
        train_group = train_group.copy().sort_values('YearMonth')

        # # Debugging checks
        print("Min values:\n", train_group[['Log_Demand', 'Log_Price', 'Log_Competitor_Price']].min())
        print("Max values:\n", train_group[['Log_Demand', 'Log_Price', 'Log_Competitor_Price']].max())
        print("Check for NaN:\n", train_group.isna().sum())

        # Ensure proper index format
        train_group['YearMonth'] = pd.to_datetime(train_group['YearMonth'])
        train_group.set_index('YearMonth', inplace=True)

        # Ensure monthly frequency without modifying actual timestamps
        train_group = train_group.asfreq('MS')

        # Extract target variable
        y_train = train_group['Log_Demand']
        # y_train = train_group['Log_Demand'].dropna()
        # print("Unique time periods:", len(y_train))

        # Extract exogenous variables, filling missing values
        exog_cols = ['Log_Price', 'Log_Competitor_Price']
        exog_train = train_group[exog_cols].ffill()  # Forward fill exog vars


        # Skip if insufficient training data
        if len(y_train) < 24:
            print("Skipping training: Insufficient data.")
            return None

        # Train SARIMAX model
        model = SARIMAX(
            y_train,
            exog=exog_train if not exog_train.empty else None,  
            order=(1, 1, 1),
            seasonal_order=(0, 1, 1, 12),  # 12-month seasonality (was 6)
            enforce_stationarity=True,  # Force stability
            enforce_invertibility=True   # Prevent large errors
        )

        start_params = [
        0.5,    # AR(1)
        0.3,    # MA(1)
        0.2,    # Seasonal MA(1)
        0.1,    # Intercept
        0.05,   # Coef for exog 1
        -0.05,  # Coef for exog 2
        1.0     # sigma² (variance of errors)
        ]
        # Fit model with more stable optimization
        fitted_model = model.fit()
        # fitted_model = model.fit(start_params = start_params, method='lbfgs', maxiter=200, disp=False)

        return fitted_model
    
    except Exception as e:
        print(f"Training failed: {str(e)}")
        return None



def calculate_elasticity(fitted_model):
    """Calculates price elasticity from fitted model"""
    try:
        # Get elasticity (coefficient of Log_Price)
        if hasattr(fitted_model, 'params') and 'Log_Price' in fitted_model.params:
            return fitted_model.params['Log_Price']
        return np.nan
    except Exception as e:
        print(f"Elasticity calculation failed: {str(e)}")
        return np.nan
    

def evaluate_model(model, test_group):
    """Evaluate model performance on test data"""
    try:
        # Clean test data
        # test_group = clean_log_values(test_group.copy())
        test_group = test_group.sort_values('YearMonth')
        
        # Prepare test data
        y_test = test_group.set_index('YearMonth')['Log_Demand']
        exog_test = test_group.set_index('YearMonth')[['Log_Price', 'Log_Competitor_Price']]
        
        # Generate forecasts
        forecast = model.get_forecast(
            steps=len(y_test),
            exog=exog_test
        )
        
        # Calculate metrics (converting back from log scale)
        pred = np.exp(forecast.predicted_mean).values  # Convert to numpy array
        actual = np.exp(y_test).values  # Convert to numpy array
        
        mae = mean_absolute_error(actual, pred)
        rmse = np.sqrt(mean_squared_error(actual, pred))
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore", RuntimeWarning)
        mape = np.mean(np.abs((actual - pred) / actual)) * 100
        
        return {
            'mae': mae,
            'rmse': rmse,
            'mape': mape,
            'actual': actual,
            'predicted': pred
        }
    
    except Exception as e:
        print(f"Evaluation failed: {str(e)}")
        return None

    
def generate_forecasts(model, train_group, steps=12):
    """Generate future forecasts with proper unit handling"""
    try:
        # Get last available data point
        last_data = train_group.sort_values('YearMonth').iloc[-1]
        
        # Create future exogenous variables
        future_exog = pd.DataFrame(
            [last_data[['Log_Price', 'Log_Competitor_Price']].values] * steps,
            columns=['Log_Price', 'Log_Competitor_Price'],
            index=pd.date_range(
                start=train_group['YearMonth'].max() + pd.offsets.MonthBegin(1),
                periods=steps,
                freq='MS'
            )
        )
        
        # Generate forecasts
        forecast = model.get_forecast(
            steps=steps,
            exog=future_exog
        ).predicted_mean
        
        # Return in original scale (only convert if model used log)
        if 'Log_Demand' in model.model.endog_names:
            return np.exp(forecast)
        return forecast
    
    except Exception as e:
        print(f"Forecast generation failed: {str(e)}")
        return None

def run_forecast_pipeline(monthly_df):
    """Complete forecasting pipeline"""
    # 1. Split data (time-based)
    train_df, test_df = train_test_split_monthly(monthly_df, test_size=0.2)
    
    detailed_results = []
    performance_metrics = []
    
    for (product, region), group in train_df.groupby(['Product_id', 'Geography']):
        # Get corresponding test data
        test_group = test_df[
            (test_df['Product_id'] == product) & 
            (test_df['Geography'] == region)
        ]
        
        # Skip if no test data
        if len(test_group) == 0:
            continue
        
        # Train model
        model = train_sarimax_model(group)
        if model is None:
            continue

        elasticity = calculate_elasticity(model)
        
        # Evaluate model
        eval_results = evaluate_model(model, test_group)
        if eval_results is None:
            continue
        
        # Generate forecasts
        forecasts = generate_forecasts(model, group)
        
        # Store results
        detailed_results.append({
            'product': product,
            'region': region,
            'model': model,
            'actual': eval_results['actual'],
            'predicted': eval_results['predicted'],
            'forecast': forecasts, 
            'elasticity': elasticity
        })
        
        performance_metrics.append({
            'product': product,
            'region': region,
            'mae': eval_results['mae'],
            'rmse': eval_results['rmse'],
            'mape': eval_results['mape']
        })
    
    return pd.DataFrame(performance_metrics), detailed_results

In [15]:
# Prepare your monthly data (ensure it has Log_Demand, Log_Price, Log_Competitor_Price columns)
monthly_df = prepare_data(df)

# Run the full pipeline
metrics_df, detailed_results = run_forecast_pipeline(monthly_df)

# Analyze results
print(metrics_df)  # Performance metrics for all product-region combinations

Min values:
 Log_Demand              1.609438
Log_Price               7.026427
Log_Competitor_Price    7.106147
dtype: float64
Max values:
 Log_Demand              5.283204
Log_Price               8.497603
Log_Competitor_Price    8.544283
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price        0
Discount                0
Log_Demand              0
Log_Price               0
Log_Competitor_Price    0
dtype: int64


c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to 

Min values:
 Log_Demand              1.386294
Log_Price               6.964136
Log_Competitor_Price    7.055063
dtype: float64
Max values:
 Log_Demand              5.093750
Log_Price               8.474912
Log_Competitor_Price    8.559708
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price        0
Discount                0
Log_Demand              0
Log_Price               0
Log_Competitor_Price    0
dtype: int64
Min values:
 Log_Demand              1.098612
Log_Price               7.181592
Log_Competitor_Price    6.889683
dtype: float64
Max values:
 Log_Demand              5.484797
Log_Price               8.471777
Log_Competitor_Price    8.639998
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price        0
Discount                0
Log_D

c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\sta

Min values:
 Log_Demand              1.098612
Log_Price               7.212294
Log_Competitor_Price    7.005490
dtype: float64
Max values:
 Log_Demand              5.303305
Log_Price               8.424859
Log_Competitor_Price    8.658127
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price        0
Discount                0
Log_Demand              0
Log_Price               0
Log_Competitor_Price    0
dtype: int64
Min values:
 Log_Demand              2.079442
Log_Price               7.182352
Log_Competitor_Price    7.176927
dtype: float64
Max values:
 Log_Demand              5.631212
Log_Price               8.494948
Log_Competitor_Price    8.579076
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price        0
Discount                0
Log_D

...

Min values:
 Log_Demand              4.795791
Log_Price               7.722530
Log_Competitor_Price    7.579658
dtype: float64
Max values:
 Log_Demand              6.556778
Log_Price               8.257602
Log_Competitor_Price    8.400333
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price        0
Discount                0
Log_Demand              0
Log_Price               0
Log_Competitor_Price    0
dtype: int64


c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


Min values:
 Log_Demand              3.218876
Log_Price               7.484743
Log_Competitor_Price    7.510846
dtype: float64
Max values:
 Log_Demand              6.621406
Log_Price               8.396381
Log_Competitor_Price    8.355508
dtype: float64
Check for NaN:
 Product_id              0
Geography               0
YearMonth               0
Demand                  0
Price                   0
Competitor_Price        0
Discount                0
Log_Demand              0
Log_Price               0
Log_Competitor_Price    0
dtype: int64


c:\Users\205379\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


                               product   region           mae          rmse  \
0             ACB(Air Circuit Breaker)  Central     30.872231     37.484725   
1             ACB(Air Circuit Breaker)     East     15.603683     25.831728   
2             ACB(Air Circuit Breaker)    North    134.160556    146.559245   
3             ACB(Air Circuit Breaker)    South     53.012011     56.070604   
4             ACB(Air Circuit Breaker)     West    133.773102    159.836872   
5        AIS(Air Insulated Switchgear)  Central    539.612270    752.554188   
6        AIS(Air Insulated Switchgear)     East    649.556756   1002.873253   
7        AIS(Air Insulated Switchgear)    North    614.678815    778.554622   
8        AIS(Air Insulated Switchgear)    South    594.804024    701.822459   
9        AIS(Air Insulated Switchgear)     West    475.504559    728.547426   
10                BBT(Busbar Trunking)  Central     99.562835    154.062136   
11                BBT(Busbar Trunking)    South     

In [18]:
print(detailed_results)

[{'product': 'ACB(Air Circuit Breaker)', 'region': 'Central', 'model': <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper object at 0x000001E0BA986450>, 'actual': array([124.,  90.,  78.,  41.]), 'predicted': array([65.43355861, 93.71311027, 96.28986749, 83.91950679]), 'forecast': 2024-02-01    32.393148
2024-03-01    62.858878
2024-04-01    36.218719
2024-05-01    64.294158
2024-06-01    26.602777
2024-07-01    43.397206
2024-08-01    58.726449
2024-09-01    79.159393
2024-10-01    33.016736
2024-11-01    31.400807
2024-12-01    29.975559
2025-01-01    35.388017
Freq: MS, Name: predicted_mean, dtype: float64, 'elasticity': np.float64(-0.07485385255987072)}, {'product': 'ACB(Air Circuit Breaker)', 'region': 'East', 'model': <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper object at 0x000001E0BA984A10>, 'actual': array([ 31.,  27.,  63., 156.,  70.,  38.,  10.]), 'predicted': array([25.71782273, 35.79937683, 64.00315015, 92.70293591, 47.78626025,
       38.61370383, 18.0

In [19]:
def save_results_to_csv(metrics_df, detailed_results, base_filename="forecast_results"):
    """
    Save forecast results to CSV files
    - Ensures proper date formatting
    - Preserves original units (no double conversion)
    - Creates clean, analysis-ready outputs
    """
    
    # 1. Save performance metrics
    metrics_df.to_csv(f"{base_filename}_metrics.csv", index=False)
    
    # 2. Save forecasts (12-month predictions)
    forecast_data = []
    elasticity_data = []
    for result in detailed_results:
        if result['forecast'] is None:
            continue
            
        # Create future dates starting next month
        last_date = result['actual'].index[-1] if isinstance(result['actual'], pd.Series) else pd.Timestamp.now()
        dates = pd.date_range(
            start=last_date + pd.offsets.MonthBegin(1),
            periods=len(result['forecast']),
            freq='MS'
        )
        
        forecast_data.append(pd.DataFrame({
            'product': result['product'],
            'region': result['region'],
            'date': dates.strftime('%Y-%m-%d'),
            'forecast': result['forecast']  # Already in original units
        }))
        # Elasticity DataFrame (simple product-region mapping)
        if 'elasticity' in result:
            elasticity_data.append({
                'product': result['product'],
                'region': result['region'], 
                'price_elasticity': result['elasticity']
            })
    
    if forecast_data:
        pd.concat(forecast_data).to_csv(f"{base_filename}_forecasts.csv", index=False)
    
    # 3. Save actual vs predicted comparisons
    comparison_data = []
    for result in detailed_results:
        if result.get('actual') is None or result.get('predicted') is None:
            continue
            
        # Get existing dates or create default range
        if isinstance(result['actual'], pd.Series):
            dates = result['actual'].index
        else:
            dates = pd.date_range(
                end=pd.Timestamp.now(),
                periods=len(result['actual']),
                freq='MS'
            )
        
        comparison_data.append(pd.DataFrame({
            'product': result['product'],
            'region': result['region'],
            'elasticity': result['elasticity'],
            'date': dates.strftime('%Y-%m-%d'),
            'actual': result['actual'],  # Already in original units
            'predicted': result['predicted']  # Already in original units
        }))
    
    if comparison_data:
        pd.concat(comparison_data).to_csv(f"{base_filename}_comparisons.csv", index=False)

    if elasticity_data:
        pd.DataFrame(elasticity_data).to_csv(f"{base_filename}_elasticity.csv", index=False)
    
    print(f"Successfully saved results to:")
    print(f"- Metrics: {base_filename}_metrics.csv")
    print(f"- Forecasts: {base_filename}_forecasts.csv")
    print(f"- Comparisons: {base_filename}_comparisons.csv")
    print(f"- Elasticity: {base_filename}_elasticity.csv")

In [20]:
# With custom filename:
save_results_to_csv(metrics_df, detailed_results, "my_product_forecasts3")

Successfully saved results to:
- Metrics: my_product_forecasts3_metrics.csv
- Forecasts: my_product_forecasts3_forecasts.csv
- Comparisons: my_product_forecasts3_comparisons.csv
- Elasticity: my_product_forecasts3_elasticity.csv


In [28]:
def create_price_discount_simulation(product_region_data, elasticity_value, product_name, region_name):
    """
    Create simulation table with:
    - 5 equidistant price buckets between min/max observed prices
    - Average demand calculated for each price bucket
    - Demand projections at 0-50% discounts (5% intervals)
    
    Args:
        product_region_data: DataFrame with historical transactions
        elasticity_value: Pre-calculated elasticity coefficient
        product_name: Product to analyze
        region_name: Region to analyze
    
    Returns:
        DataFrame with simulation scenarios
    """
    
    # Filter for product-region
    data = product_region_data[
        (product_region_data['Product_id'] == product_name) & 
        (product_region_data['Geography'] == region_name)
    ].copy()
    
    if data.empty:
        print(f"No data for {product_name} in {region_name}")
        return None
    
    # Create 4 price buckets
    min_price = data['Price'].min()
    max_price = data['Price'].max()
    price_buckets = np.linspace(min_price, max_price, 5)
    
    # Calculate average demand per price bucket
    data['price_bucket'] = pd.cut(data['Price'], bins=price_buckets, include_lowest=True)
    bucket_stats = data.groupby('price_bucket').agg(
        avg_demand=('Demand', 'mean'),
        price_midpoint=('Price', lambda x: (x.min() + x.max())/2)
    ).reset_index()
    
    # Generate all discount scenarios (0-50% in 5% steps)
    discount_rates = np.arange(0, 55, 5)
    
    # Build simulation scenarios
    simulation_results = []
    for _, bucket in bucket_stats.iterrows():
        for discount in discount_rates:
            new_demand = calculate_new_demand(
                demand=bucket['avg_demand'],
                elasticity=elasticity_value,
                discount=discount
            )
            
            simulation_results.append({
                'product': product_name,
                'region': region_name,
                'price_bucket': f"{bucket['price_bucket'].left:.2f}-{bucket['price_bucket'].right:.2f}",
                'price_midpoint': round(bucket['price_midpoint'], 2),
                'discount_pct': discount,
                'original_avg_demand': round(bucket['avg_demand'], 2),
                'predicted_demand': round(new_demand, 2),
                'demand_change_pct': round((new_demand - bucket['avg_demand'])/bucket['avg_demand']*100, 1),
                'elasticity_used': round(elasticity_value, 3)
            })
    
    return pd.DataFrame(simulation_results)

def calculate_new_demand(demand, elasticity, discount):
    """Calculate demand response to price changes"""
    discount = np.clip(discount, 0, 50)
    if elasticity is None or np.isnan(elasticity):
        elasticity = -0.2
    return demand * np.exp(elasticity * np.log(1 - discount/100))

In [29]:
# Load your data and elasticity
elasticity_df = pd.read_csv("my_product_forecasts3_elasticity.csv")
product_elasticity = elasticity_df[
    (elasticity_df['product'] == "RMU(Ring Main Unit)") & 
    (elasticity_df['region'] == "West")
]['price_elasticity'].values[0]

# Generate simulation
simulation = create_price_discount_simulation(
    product_region_data=monthly_df,
    elasticity_value=product_elasticity,
    product_name="RMU(Ring Main Unit)",
    region_name="West"
)

# Save results
simulation.to_csv("rmu_west_simulation.csv", index=False)

C:\Users\205379\AppData\Local\Temp\ipykernel_7996\403471707.py:35: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_stats = data.groupby('price_bucket').agg(


In [30]:
def calculate_optimal_discounts(simulation_df):
    """
    Calculate profit-optimizing discount at each price point
    Returns DataFrame with optimal discount scenarios
    """
    
    # Calculate revenue for each scenario
    simulation_df['revenue'] = (
        simulation_df['price_midpoint'] * 
        (1 - simulation_df['discount_pct']/100) * 
        simulation_df['predicted_demand']
    )
    
    # Find discount that maximizes revenue for each price bucket
    optimal_discounts = (
        simulation_df.loc[simulation_df.groupby('price_bucket')['revenue'].idxmax()]
        [['product', 'region', 'price_bucket', 'price_midpoint', 
          'discount_pct', 'predicted_demand', 'revenue']]
        .sort_values('price_midpoint')
        .rename(columns={
            'discount_pct': 'optimal_discount_pct',
            'predicted_demand': 'optimal_demand',
            'revenue': 'max_revenue'
        })
    )
    
    return optimal_discounts

In [31]:
## sign of elasticity - 
## 

In [26]:

def save_optimal_discounts(simulation_df, filename="optimal_discounts.csv"):
    """Calculate and save optimal discount scenarios"""
    optimal_df = calculate_optimal_discounts(simulation_df)
    
    # Format output
    optimal_df['price_midpoint'] = optimal_df['price_midpoint'].round(2)
    optimal_df['max_revenue'] = optimal_df['max_revenue'].round(2)
    
    # Save to CSV
    optimal_df.to_csv(filename, index=False)
    print(f"Saved optimal discounts to {filename}")
    return optimal_df

In [27]:
# 2. Calculate and save optimal discounts
optimal_discounts = save_optimal_discounts(
    simulation,
    filename="rmu_west_optimal_discounts.csv"
)

# Display sample results
print(optimal_discounts.head())

Saved optimal discounts to rmu_west_optimal_discounts.csv
                product region     price_bucket  price_midpoint  \
0   RMU(Ring Main Unit)   West  2537.96-2784.72         2659.17   
11  RMU(Ring Main Unit)   West  2784.72-3031.48         2908.06   
22  RMU(Ring Main Unit)   West  3031.48-3278.24         3180.61   
33  RMU(Ring Main Unit)   West  3278.24-3525.00         3404.03   

    optimal_discount_pct  optimal_demand  max_revenue  
0                      0         1168.08   3106123.29  
11                     0         1365.86   3972002.83  
22                     0         1284.83   4086543.15  
33                     0         1434.75   4883932.04  
